In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [31]:
# Load the dataset
df = pd.read_csv('data/youmultihatred.csv')

In [32]:
df.columns

Index(['Text', 'IsToxic', 'IsAbusive', 'IsProvocative', 'IsObscene',
       'IsHatespeech', 'IsRacist'],
      dtype='object')

In [35]:
import re

# Convert text to lowercase, remove unnecessary blank spaces, URLs, and special characters
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)  # remove URLs
    text = re.sub(r'\s+', ' ', text).strip()  # remove extra spaces
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # remove special characters
    return text

df['Text'] = df['Text'].apply(preprocess_text)

In [36]:

# Preprocess the data
texts = df['Text'].values
labels = df[['IsToxic', 'IsAbusive', 'IsProvocative', 'IsObscene', 'IsHatespeech', 'IsRacist']].values

# Tokenize the text
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

# Pad the sequences
data = pad_sequences(sequences, maxlen=100)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Build the model
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=100))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(6, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Predict the probabilities
predictions = model.predict(X_test)


Epoch 1/10
20/20 [==============================] - 5s 135ms/step - loss: 0.5487 - accuracy: 0.7141 - val_loss: 0.4653 - val_accuracy: 1.0000
Epoch 2/10
20/20 [==============================] - 2s 97ms/step - loss: 0.4832 - accuracy: 0.8609 - val_loss: 0.4623 - val_accuracy: 1.0000
Epoch 3/10
20/20 [==============================] - 2s 93ms/step - loss: 0.4819 - accuracy: 0.9000 - val_loss: 0.4614 - val_accuracy: 1.0000
Epoch 4/10
20/20 [==============================] - 2s 93ms/step - loss: 0.4763 - accuracy: 0.9187 - val_loss: 0.4593 - val_accuracy: 1.0000
Epoch 5/10
20/20 [==============================] - 2s 95ms/step - loss: 0.4623 - accuracy: 0.8703 - val_loss: 0.4640 - val_accuracy: 0.9688
Epoch 6/10
20/20 [==============================] - 2s 95ms/step - loss: 0.4050 - accuracy: 0.8125 - val_loss: 0.4561 - val_accuracy: 1.0000
Epoch 7/10
20/20 [==============================] - 2s 95ms/step - loss: 0.3108 - accuracy: 0.8047 - val_loss: 0.4900 - val_accuracy: 0.9563
Epoch 8/10
2

In [37]:
# Evaluate the model on the training data
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
print(f'Training Loss: {train_loss:.4f}')
print(f'Training Accuracy: {train_accuracy:.4f}')

# Evaluate the model on the testing data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Testing Loss: {test_loss:.4f}')
print(f'Testing Accuracy: {test_accuracy:.4f}')

overfitting_percentage = ((train_accuracy - test_accuracy) / train_accuracy) * 100

print(f'Overfitting Percentage: {overfitting_percentage:.2f}%')

# Check for overfitting
if train_accuracy > test_accuracy:
    print("The model is overfitting.")
else:
    print("The model is not overfitting.")

Training Loss: 0.2445
Training Accuracy: 0.5400
Testing Loss: 0.7143
Testing Accuracy: 0.7300
Overfitting Percentage: -35.19%
The model is not overfitting.


In [ ]:
# Sample text
input_text = 'You freaking scumbag! I hate you.'

# Preprocess text
# Convert to sequence
sequence = tokenizer.texts_to_sequences([input_text])

# Pad sequence
padded_sequence = pad_sequences(sequence, maxlen=100)

# Predict
probabilities = model.predict(padded_sequence)

# Print results
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
print("Prediction probabilities:")
for label, prob in zip(labels, probabilities[0]):
    print(f"{label}: {prob:.2f}")


1/1 [==============================] - 0s 70ms/step
Prediction probabilities:
toxic: 0.95
severe_toxic: 0.70
obscene: 0.20
threat: 0.04
insult: 0.28
identity_hate: 0.24


In [39]:
model.save('models/text_classification_model.h5')

In [41]:
import tf2onnx
import onnx

# Convert the Keras model to ONNX format
onnx_model, _ = tf2onnx.convert.from_keras(model)

# Save the ONNX model to a file
onnx.save(onnx_model, 'models/text_classification_model.onnx')

In [42]:
import pickle

# Save tokenizer
with open('models/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)